Primera version del modelo sin actualizar

# 1. DataLoader

In [1]:
import torch
import torchaudio
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split
from Data_Paths_Model import *
import numpy as np
import torchaudio.transforms as T
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
from torchvision import transforms

/home/jose/.local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/jose/.local/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [10]:
class AudiosDataset(Dataset):
    def __init__(self, labels_df, dataset_df, image_sizes=(220, 220), n_fft=2048, hop_length=512):
        """
        Initialize the Audios Dataset class.

        Args:
            labels_df (pandas.DataFrame): DataFrame containing audio labels.
            dataset_df (pandas.DataFrame): DataFrame containing audio files and their corresponding Mel spectrograms.
            image_sizes (tuple, optional): Tuple containing the size of the image (height, width). Default is (220, 220).
            n_fft (int, optional): Number of samples per frame for Fourier Transform. Default is 2048.
            hop_length (int, optional): Number of samples between frames. Default is 512.
        """
        self.labels_df = labels_df
        self.dataset_df = dataset_df
        self.image_sizes = image_sizes
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.normalize = transforms.Normalize(mean=0, std=1)

    def __len__(self):
        """
        Returns:
            int: Length of the dataset.
        """
        return len(self.labels_df)

    def __getitem__(self, idx):
        """
        Args:
            idx (int): Index of the item.

        Returns:
            tuple: Tuple containing the Mel spectrogram and its corresponding label.
        """
        # Get the audio path and label from the labels DataFrame
        audio_path = self.labels_df.loc[idx, 'audio_path']
        label = self.labels_df.loc[idx, 'forgery_type_index']
        
        # Find the corresponding row in the dataset DataFrame
        dataset_row = self.dataset_df[self.dataset_df['audio_path'] == audio_path].iloc[0]
        
        # Extract the Mel spectrogram
        mel_spectrogram_str = dataset_row['spectogram_of_audio']
        mel_spectrogram = np.array(eval(mel_spectrogram_str)) 
        
        # Convert the NumPy array to a PyTorch tensor
        mel_spectrogram_tensor = torch.tensor(mel_spectrogram)  
        
        # Resize the images
        resize = T.Resize(self.image_sizes)
        mel_spectrogram_resized = resize(mel_spectrogram_tensor)

        # Normalize the spectrogram data
        mel_spectrogram_tensor = self.normalize(mel_spectrogram_resized)

        # Apply data augmentation
        # aug_mel_spectrogram = self.spectro_augment(mel_spectrogram_resized)
        
        return mel_spectrogram_tensor, label

    @staticmethod
    def spectro_augment(spec, max_mask_pct=0.1, n_freq_masks=1, n_time_masks=1, noise_std=0.1):
        """
        SpecAugment: data augmentation technique for spectrograms. 
        It applies frequency and time masks to the spectrogram to avoid overfitting and help the model generalize better. 
        the model to generalize better. This data augmentation process helps to improve the performance and generalizability of the spectrogram. 
        performance and generalization capability of convolutional neural network models trained on audio data. 
        trained on audio data.

        Args:
            spec (torch.Tensor): Input spectrogram.
            max_mask_pct (float, optional): Maximum percentage of the spectrogram to mask.
            n_freq_masks (int, optional): Number of frequency masks to apply.
            n_time_masks (int, optional): Number of time masks to apply.
            noise_std (float, optional): Standard deviation of the Gaussian noise to add.

        Returns:
            torch.Tensor: Augmented spectrogram.
        """
        aug_spec = spec

        # Apply frequency masking
        freq_mask_param = int(max_mask_pct * aug_spec.size(1))
        for _ in range(n_freq_masks):
            aug_spec = torchaudio.transforms.FrequencyMasking(freq_mask_param)(aug_spec)

        # Apply time masking
        time_mask_param = int(max_mask_pct * aug_spec.size(2))
        for _ in range(n_time_masks):
            aug_spec = torchaudio.transforms.TimeMasking(time_mask_param)(aug_spec)

        # Add Gaussian noise
        noise = torch.randn_like(aug_spec) * noise_std
        aug_spec = aug_spec + noise

        return aug_spec

In [11]:
# Load the labels and audios DataFrame
labels_df = pd.read_csv(LABELS_DATASET_PATH)
dataset_df = pd.read_csv(AUDIOS_DATASET_PATH)

# Create the Sound Dataset instance
sound_dataset = AudiosDataset(labels_df, dataset_df)

# Split the dataset into train and test
train_size = int(0.8 * len(sound_dataset))
test_size = len(sound_dataset) - train_size
train_dataset, test_dataset = random_split(sound_dataset, [train_size, test_size])

# Dataloaders
train_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=16, shuffle=False)

# 2. Train the models

Some standard values for all models

In [12]:
num_epochs = 10

## 2.1. ResNet

In [13]:
# Define the pretrained ResNet model
model = models.resnet18(pretrained=True)

# Modify the last layer to have 3 output classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")



/home/jose/src/TFG/LabelDataset/DeepLearningForgeryAudioDetection/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jose/src/TFG/LabelDataset/DeepLearningForgeryAudioDetection/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


SyntaxError: invalid syntax. Perhaps you forgot a comma? (<string>, line 1)

Evaluate the model

In [ ]:
# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {100 * correct / total}%")